<a href="https://colab.research.google.com/github/ElaYJ/Study_Deep_Learning/blob/main/Lecture/52_GPT_HF_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Train 학습

- Fine Tuning 미세 조정 과정

    - Loss Function 생성

    - Optimizer 만들기

    - backward를 통해 weights를 업데이트

- 실무에서 구현 가능한 예제

<br></br>

## Pipeline 내부 실행 과정

- https://wikidocs.net/166795

- 파이프라인은 전처리(preprocessing), 모델로 입력 전달 및 후처리(postprocessing)의 3단계를 한번에 실행

	- preprocess --> MODEL --> post process

	- 이 3개의 과정을 pipeline을 통해 한번에~

	<img src="https://github.com/ElaYJ/supplement/assets/153154981/1c4cf46c-883c-40f6-a60e-ca32c015d4fd" width="67%"><br></br>


### - tokenizer

- 간단한 tokenizer 구현

- 모델이 사용한 tokenizer를 쓰지 않으면 성능이 많이 떨어진다.

- tokenizer를 이용한 모든 전처리(preprocessing)는 모델이 사전 학습(pretraining)될 때와 정확히 동일한 방식으로 수행되어야 하므로

	먼저 [Model Hub](https://huggingface.co/models)에서 해당 정보를 다운로드해야 합니다.

	이를 위해 AutoTokenizer 클래스와 from_pretrained() 메서드를 사용는 것이다.

</br>

- `sentiment-analysis` 파이프라인의 디폴트 체크포인트(default checkpoint)는 `distilbert-base-uncased-finetuned-sst-2-english`이다.

	(이 모델에 대한 model card는 https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english에서 확인 가능하다.)

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
# model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [1]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [2]:
raw_inputs = [
    "I've been waiting for a huggingface course my whole life",
    "i hate this so much!",
]

# 토크나이저가 반환하는 텐서의 유형(PyTorch, TensorFlow 또는 일반 NumPy)을 지정하려면 return_tensors 인수(argument)를 사용하면 된다.
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt') # 'pt' - pytorch

In [3]:
# 토크나이저(tokenizer)를 생성한 후 이 토크나이저에 문장을 입력하여 모델에 바로 전달할 수 있는 파이썬 딕셔너리(dictionary) 정보를 구할 수 있다!
# 이후 해야할 일은 input IDs 리스트를 텐서(tensors)로 변환하는 것뿐이다.
# Transformer 모델은 텐서(tensor) 입력만 받는다.

inputs

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}

In [4]:
inputs['input_ids']
#--> 101은 시작을 나타내는 token
#--> 1045은 'I'를 나타내는 token
#--> 0으로 padding이 됨.

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]])

In [5]:
inputs['attention_mask']
#--> 문장의 길이가 다르므로 input 차원을 맞추기 위해 의미없는 0으로 padding 되어야 한다.

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])

### - pretrained model

- `AutoModel`

In [6]:
from transformers import AutoModel

# 모델 불러오기
# 토크나이저와 동일한 방식으로 사전 학습된 모델(pretrained model)을 다운로드할 수 있다.
model = AutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [7]:
# 입력이 주어지면 특성(feature) 이라고도 불리는 hidden states 를 출력한다.
# Transformer 모델에 의해서 수행된 해당 입력의 문맥적 이해(contextual understanding) 결과를 나타내는 고차원 벡터(high-dimensional vector)를 가져온다.

outputs = model(**inputs)
outputs

BaseModelOutput(last_hidden_state=tensor([[[-0.1378,  0.1974,  0.6768,  ..., -0.2884,  0.4254,  0.1880],
         [ 0.2657,  0.5706,  0.3957,  ..., -0.1252,  0.4597,  0.1770],
         [ 0.7825,  0.1824,  0.2876,  ...,  0.2613, -0.0953, -0.5923],
         ...,
         [ 0.1757,  0.3456,  0.3675,  ..., -0.5196,  0.6726, -0.2503],
         [ 0.0929,  0.4920,  0.3286,  ..., -0.3443,  0.4261, -0.0208],
         [ 0.2093,  0.4148,  0.4770,  ...,  0.3803,  0.4811, -0.5183]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.2319,  0.8268, -0.0312,  ..., -0.0764, -0.8509, -0.1043],
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861],
         [-0.3338,  0.9674, -0.0729,  ..., -0.1952, -0.8181, -0.0634]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [8]:
# Transformers 모델의 출력은 namedtuple 또는 딕셔너리(dictionary)처럼 동작한다.
# 요소에 접근하기 위해서 속성 또는 키(outputs["last_hidden_state"])를 사용할 수 있다.
# 찾고 있는 항목이 어디에 있는지 정확히 알고 있는 경우 인덱스(outputs[0])로도 액세스할 수 있다.

outputs.last_hidden_state.shape
#--> ([batch size=2, sequence length=16, hidden size=768])
# - 배치 크기(Batch size): 한 번에 처리되는 시퀀스(sequence)의 개수
# - 시퀀스 길이(Sequence length): 시퀀스 숫자 표현의 길이
# - 은닉 크기(Hidden size): 각 모델 입력의 벡터 차원

torch.Size([2, 15, 768])

- Model hidden output

    - ForCasulLM

    - FormaskedLM

    - ForQuestionAnswering

    - etc.

- `AutoModelForSequenceClassification`

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [10]:
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.4683,  1.5105],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
outputs.logits
#--> 우리 모델은 첫 번째 문장에 대해 [-1.4683,  1.5105], 두 번째 문장에 대해 [4.1692, -3.3464]를 예측했다.
#--> 이는 확률이 아니라 모델의 마지막 계층에서 출력된 정규화되지 않은 원시 점수(logits)이다.
#--> 이들 값을 확률로 변환하려면 SoftMax 계층을 통과해야 한다.

tensor([[-1.4683,  1.5105],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)

In [12]:
outputs.logits.shape #--> 2 x 2 (2개의 문장과 2개의 label)

torch.Size([2, 2])

### - post processing

In [13]:
import torch

# 출력(outputs) 후처리
# 모델의 출력값 자체로 의미를 가지지 못한다.
prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
prediction

tensor([[4.8393e-02, 9.5161e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)

In [14]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

<br></br>

## Models

- https://wikidocs.net/166799

- 모델 생성과 사용 방법 알아보기 <br></br>


### - BERT

- 트랜스포머 모델 생성하기

- BERT 모델을 초기화하기 위해 가장 먼저 해야 할 일은 설정(configuration) 객체를 로드하는 것이다.

In [15]:
from transformers import BertConfig, BertModel

config = BertConfig() # config(설정) 생성
config
#--> hidden_size 속성은 hidden_states 벡터의 크기를 정의한다.
#--> num_hidden_layers 는 Transformer 모델의 계층(layers) 수를 정의

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.38.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [16]:
model = BertModel(config)
model
#--> 기본 설정(configuration)에서 모델을 생성하면 해당 모델을 임의의 값으로 초기화된다.
#--> 즉 모델이 학습된 값이 아니라 무작위로 초기화되는 것이다.

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [17]:
config.hidden_size

768

In [18]:
# config 값을 조정해 fine tuning 할 수 있다.

config.hidden_size = 48

model = BertModel(config)
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 48, padding_idx=0)
    (position_embeddings): Embedding(512, 48)
    (token_type_embeddings): Embedding(2, 48)
    (LayerNorm): LayerNorm((48,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=48, out_features=48, bias=True)
            (key): Linear(in_features=48, out_features=48, bias=True)
            (value): Linear(in_features=48, out_features=48, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=48, out_features=48, bias=True)
            (LayerNorm): LayerNorm((48,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
   

In [19]:
# 이미 학습된 transformer 모델을 from_pretrained() 메서드를 사용해 로드 할 수 있다.

model = BertModel.from_pretrained("bert-base-cased")
model

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [20]:
# train...
# 학습한 후 모델을 저장할 수 있다.

model.save_pretrained("./test")

In [21]:
ls test

config.json  model.safetensors


In [ ]:
# config.json 파일 내용을 보면 모델 아키텍처(model architecture)를 구축하는 데 필요한 다양한 속성들을 볼 수 있다.
# pytorch_model.bin 파일은 state dictionary 라고도 부르며 여기에는 모델의 모든 가중치가 저장되어 있다.
# 이 두 파일은 함께 사용된다.
# 설정 객체(configuration objects)는 모델의 아키텍처(model architecture)를 파악하는데 필요한 반면,
# 모델 가중치는 모델의 매개변수(parameters)이다.

config.json  pytorch_model.bin

</br>

### - tokenizer

- https://wikidocs.net/166796

- Split on spaces : 공백을 기반으로 토큰화

- Split on punctuation : 구두점을 기반으로 토큰화

In [22]:
# 공백(단어) 기반 토큰화 예제

tokenized_text = "Jim Henson was puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'puppeteer']


- 토큰이 중요한 이유

    - '[UNK]' : unknown

    - unknown token이 많을수록 모델을 fine tuning 할 때 애로사항이 많이 생길 수 있다.

    - 그래서 unknown token을 어떻게 없앨 수 있을까가 해결해야 할 큰 숙제이다.

    - 예를 들어, 일반적인 책이나 wiki pedia를 데이터셋으로 학습한 모델에 twitter를 input data로 사용할 때 트위터에 많은 '#~'를 텍스트로 처리할 것인지, 아니면 지울 것인지, 어떻게 처리할 것인지 결정해야 한다. 이모티콘도 string올 표현되는 바이트이므로 어떻게 처리할 지에 대해 고민해야한다. 너무 많은 unknown token이 존재하게 되면 제대로된 evaluation이 불가능하게 된다.

</br>

- BERT는 subword(접미사, 접두어, 등)으로 tokenize 된다.

    - WordPiece (BERT에 사용됨)

    - Byte-level BPE (GPT-2에 사용됨)

In [23]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [24]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
# 토크나이저를 저장하는 것은 모델을 저장하는 것과 동일하다.

tokenizer.save_pretrained("saving_folder")

('saving_folder/tokenizer_config.json',
 'saving_folder/special_tokens_map.json',
 'saving_folder/vocab.txt',
 'saving_folder/added_tokens.json')

- 인코딩 Encoding

	- 텍스트를 숫자로 변환하는(translating text to numbers) 과정을 인코딩(encoding) 이라고 한다.

	- 인코딩(encoding)은 토큰화와 입력 식별자(input IDs)로의 변환이라는 2단계 프로세스로 수행된다.

In [26]:
from transformers import AutoTokenizer

# 1. 토큰화 작업

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens) #--> '##former' 이런 것들이 존재하게 된다.

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [27]:
# 만약 전자기기 메뉴얼 데이터셋이라면 BERT와 같이 tokenize 되는 모델은 맞지 않는 것이 된다.

sequence = "Using a Transformer network is menual, KT-12327"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'menu', '##al', ',', 'K', '##T', '-', '123', '##27']


In [28]:
# 2. 토큰을 입력 식별자로 변환 (From tokens to input IDs)

ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 13171, 1348, 117, 148, 1942, 118, 13414, 24458]


- 디코딩 Decoding

	- 변환된 입력 식별자(input IDs)를 이용해서 어휘집(vocabulary)에서 해당 문자열을 찾는다.

In [29]:
# 인덱스로 디코더도 가능하다.

tokenizer.decode([7993, 170, 13809, 23763, 2443, 1110, 13171, 1348, 117, 148, 1942, 118, 13414, 24458])

'Using a Transformer network is menual, KT - 12327'

</br>

### - batch

- 다중 시퀀스 처리 : 모델(model)은 입력의 배치(batch) 형태를 요구한다.

	- Batching 이란 모델을 통해 한번에 여러 문장을 입력하는 동작이다.

	- 배치(batch) 처리를 통해서 모델이 여러 문장을 동시에 입력받을 수 있도록 할 수 있다.

In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
model(input_ids) #--> This line will fail. why? Shape이 맞지 않게 때문이다. --> Matix로 만들어 줘야 한다.
				 #--> 오류가 발생한 코드에서 input_ids에 새로운 차원을 하나 추가하면 해결된다.

IndexError: too many indices for tensor of dimension 1

In [31]:
input_ids, input_ids.shape

(tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]),
 torch.Size([14]))

In [32]:
tokenized_inputs = tokenizer(sequence, return_tensors='pt')
print(tokenized_inputs['input_ids']) #--> 이렇게 하면 자동으로 행과 열의 형태로 만들어준다.

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [33]:
tokenized_inputs['input_ids'].shape
#--> batch 1개와 16개의 token
#--> 이런 식으로 batch 기반의 형태로 만들어 줘야 한다.
#--> tokenizer() 함수를 이용하면 이러한 행렬 형태로 만들어 주는데,
# tokens = tokenizer.tokenize(sequence)
# ids = tokenizer.convert_tokens_to_ids(tokens)
#--> 이런 식으로 따로 해주면 리스트 형이 만들어 진다.

torch.Size([1, 16])

In [34]:
checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids]) #--> input_ids에 새로운 차원을 하나 추가
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits) #--> 0과 1의 확률 --> 긍정

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [35]:
tokens = tokenizer.tokenize(sequence)
print(tokens)

['i', "'", 've', 'been', 'waiting', 'for', 'a', 'hugging', '##face', 'course', 'my', 'whole', 'life', '.']


In [36]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


</br>

### - padding

- 모델의 입력 형태인 텐서(tensor)는 항상 그 형태가 직사각형 모양이어야 한다.

- 입력 식별자(input IDs) 리스트를 텐서로 직접 변환할 수 없는 문제를 해결하기 위해 일반적으로 입력을 채운다(padding).

- 한 문장이 아닌 여러 개의 문장일 때는 어떤 shape으로 input해줘야 할까?

- batch size가 2이고 문장의 길이가 3인 데이터셋을 넣었을 때 어떻게 될 것인가??

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200],
]
#--> 리스트(혹은 이중 리스트)는 텐서로 변환할 수 없다.

In [37]:
batched_ids = [
    [200,200,200],
    [200,200, tokenizer.pad_token_id],
]
#--> 패딩 토큰(padding token)의 식별자(ID)는 tokenizer.pad_token_id에 지정되어 있습니다.

attention_mask = [
    [1, 1, 1],
    [1, 1, 0]
]
#--> 어텐션 마스크(attention mask)는 0과 1로 채워진 입력 식별자(input IDs) 텐서(tensor)와 형태가 정확하게 동일한 텐서(tensor)이다.
#--> 1은 해당 토큰에 주의를 기울여야 함을 나타내고 0은 해당 토큰을 모델의 어텐션 레이어(attention layers)에서 무시해야 함을 나타낸다.

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


In [38]:
tokenizer.decode([200, tokenizer.pad_token_id]) #--> 200은 안쓰는 ID이다.

'[unused195] [PAD]'

- 다양한 모드에 따라 패딩(padding) 처리를 할 수 있다.

In [39]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

In [40]:
# 해당 시퀀스를 리스트 내의 최대 시퀀스 길이까지 패딩(padding) 합니다.
model_inputs = tokenizer(sequences, padding="longest")
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [41]:
# 시퀀스를 모델 최대 길이(model max length)까지 패딩(padding) 합니다.
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [42]:
# 지정된 최대 길이까지 시퀀스를 패딩(padding) 합니다.
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0]]}

- 시퀀스를 자를 수도 있다.

In [43]:
# 모델 최대 길이(model max length)보다 긴 시퀀스를 자릅니다.
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [44]:
# 지정된 최대 길이보다 긴 시퀀스를 자릅니다.
model_inputs = tokenizer(sequences, max_length=8, truncation=True)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [45]:
# 주요 API로 다중 시퀀스(패딩, padding!), 매우 긴 시퀀스(절단, truncation!), 여러 유형의 텐서를 처리하는 방법

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
print(output)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


<br></br>

## MRPC

- MRPC(Microsoft Research Paraphrase Corpus) 데이터셋을 예제로 사용한다.

- 이 데이터셋은 5,801건의 문장 쌍으로 구성되어 있으며 각 문장 쌍의 관계가 의역(paraphrasing) 관계인지 여부를 나타내는 레이블이 존재한다.</br>
    (즉, 두 문장이 동일한 의미인지 여부가 label로 존재)

- 이 데이터셋은 10개의 데이터셋으로 구성된 [GLUE 벤치마크](https://gluebenchmark.com/) 중 하나이다.

- GLUE 벤치마크는 10가지 텍스트 분류 작업을 통해서 기계학습 모델의 성능을 측정하기 위한 학술적 벤치마크 데이터 집합이다.

- 데이터셋의 규모가 그리 크지 않기 때문에 학습 과정을 쉽게 실험할 수 있습니다.

- https://wikidocs.net/166801

</br>

### - load dataset

- data를 load할 수 있으면 train(학습)도 가능하다.

- dataload --> model --> optimizer --> loss func. --> training

- load_dataset 구현 : install한 datasets에서 가능하다.

In [2]:
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

# 데이터셋 불러오기
# load_dataset 명령은 기본적으로 ~/.cache/huggingface/dataset에 데이터셋을 다운로드하고 임시저장(cache)한다.
# HF_HOME 환경 변수를 설정하여 캐시 폴더를 변경할 수 있다.
raw_datasets = load_dataset("glue", "mrpc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
# 학습(training), 검증(validation) 및 평가(test) 집합이 저장된 DatasetDict 객체를 얻을 수 있다.
# 학습 집합(training set)에는 3,668개의 문장 쌍,
# 검증 집합(validation set)에는 408개,
# 평가 집합(test set)에는 1,725개의 문장 쌍이 있다.

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
raw_datasets['train']

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})

In [6]:
raw_datasets['train'].features
#--> 레이블(label)은 ClassLabel 타입이고 레이블 이름에 대한 정수 매핑은 names 폴더에 저장되어 있다.
#--> 0은 not_equivalent를 의미하고, 1은 equivalent를 나타낸다.

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [7]:
raw_train_dataset = raw_datasets['train']
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [8]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

</br>

### - tokenizer

- 데이터 전처리 과정의 자동화

In [9]:
from transformers import AutoTokenizer

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [58]:
# 두 개의 시퀀스를 각각의 문장으로 모델에 별도의 매개변수로 전달하여 두 문장이 의역인지 아닌지에 대한 예측을 얻을 수는 없다.

tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])
print(len(tokenized_sentences_1))
print(len(tokenized_sentences_2))

3
3


In [60]:
tokenized_sentences_1[0]

Encoding(num_tokens=25, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [56]:
# tokenizer는 다음과 같이 한 쌍의 시퀀스를 가져와 BERT 모델이 요구하는 입력 형태로 구성할 수 있다.

inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [61]:
# token_type_ids는 전체 입력(input_ids)의 어느 부분이 첫 번째 문장이고 어느 것이 두 번째 문장인지 모델에 알려준다.
# "[CLS] 문장1 [SEP]"에 해당하는 입력 부분은 token_type_id가 0이고 "문장2 [SEP]"에 해당하는 다른 부분은 모두 1이다.

tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

In [10]:
# 이 raw_datasets을 맵핑해준다.
# 먼저 tokenize_function()을 정의해서 'sentence1', 'sentence2'라는 입력 피쳐를 맵핑시켜준다.
# 두 시퀀스를 쌍(pair)으로 처리(단일 매개변수로 처리)하고 적절한 전처리를 적용해야 한다.

def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

In [11]:
# 특정 데이터를 dataset 객체로 유지하기 위해 Dataset.map() 메서드를 사용한다.
# map 메서드 호출에서 batched=True를 사용해 함수가 각 요소에 개별적으로 적용되지 않고 데이터셋의 하부집합,
# 즉 각 배치(batch) 내에 존재하는 모든 요소들에 한꺼번에 적용된다. 이 방법은 더 빠른 전처리를 가능하게 한다.

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [13]:
# 필요없는 컬럼을 삭제하고 컬럼명을 변경한다.

tokenized_datasets = tokenized_datasets.remove_columns(['sentence1','sentence2','idx'])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets['train'].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

</br>

### - dynamic padding

- 동적 패딩

- 샘플들을 함께 모아서 지정된 크기의 배치(batch)로 구성하는 역할을 하는 함수를 콜레이트 함수(collate function) 라고 한다.

- 기본값은 단순히 샘플들을 PyTorch 텐서로 변환하고 결합하는 함수이다.

- 대상 샘플들이 리스트, 튜플 혹은 딕셔너리면 재귀적으로 이 작업이 수행된다.

- Transformers 라이브러리는 DataCollatorWithPadding을 통해 이러한 기능을 제공한다.

- 이 함수는 DataLoader를 빌드(build)할 때 전달할 수 있는 매개변수이다.

In [16]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# batch에 대한 것만 있고 padding에 대한 정보가 없으므로 정의해줄 필요가 있다.
#--> data_collator(collator:대조자)가 데이터셋에 대한 padding을 자동으로 해준다.

In [ ]:
# # summary code

# from datasets import load_dataset
# from transformers import AutoTokenizer, DataCollatorWithPadding

# raw_datasets = load_dataset("glue", "mrpc")
# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

</br>

### - dataloader

- https://wikidocs.net/166803

In [17]:
from torch.utils.data import DataLoader

# 최종 데이터 로더
train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets['validation'], batch_size=8, collate_fn=data_collator
)

In [18]:
# 데이터 확인
# 데이터 처리에 오류가 없는지 빠르게 확인하기 위해 다음과 같이 배치(batch)를 검사할 수 있다.

for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 60]),
 'token_type_ids': torch.Size([8, 60]),
 'attention_mask': torch.Size([8, 60])}

In [19]:
{k: v for k, v in batch.items()}

{'labels': tensor([0, 0, 1, 1, 1, 1, 0, 1]),
 'input_ids': tensor([[  101, 23268, 12662,  2001,  9687,  2000,  3731,  2966,  2415,  2007,
           2512, 15509,  1011,  8701,  6441,  1012,   102,  2119,  2020,  2579,
           2000,  3731,  2966,  2415,  1012,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  1996,  2382,  1011,  2095,  5416,  2149, 14142, 22123,  1027,
          25269, 13537,  2403,  1013,  3590,  2005,  1037, 10750,  1997,  1018,
           1012,  2656,  3867,  2013,  1018,  1012,  2603,  3867,  1012,   102,
           1996,  2382,  1011,  2095,  5416,  2149, 14142, 22123,  1027, 25269,
           2439,  2385,  1013,  3590,  1010,  2635,  2049, 10750,  2000,  1018,
           1012,  2322,  3867,  2013,  1018,  1012,  2324,  3

-- 여기까지 데이터 전처리 완료 --

-----

</br>

### - pretrained model

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2) # Negative, Positive로 label 2개

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)
#--> 모든 Transformers 모델은 매개변수에 labels이 포함되어 있다면 손실(loss)과 함께 logit값도 반환한다.
#	 (logit값: batch내 각 입력에 대해 logit값이 2개이므로 크기가 8 x 2인 텐서)

tensor(0.6788, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [22]:
outputs.logits

tensor([[-0.3790,  0.5017],
        [-0.4552,  0.5423],
        [-0.4462,  0.5275],
        [-0.4485,  0.5382],
        [-0.4576,  0.5486],
        [-0.4467,  0.5488],
        [-0.4577,  0.5446],
        [-0.4551,  0.5464]], grad_fn=<AddmmBackward0>)

</br>

### - optimizer

In [23]:
from transformers import AdamW

# 학습률 스케줄러(learning rate scheduler)는 최대값(5e-5)에서 0까지 선형 감쇠(linear decay)한다.
optimizer = AdamW(model.parameters(), lr=5e-5, no_deprecation_warning=True)
# FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version.
# Use the PyTorch implementation torch.optim.AdamW instead,
# or set `no_deprecation_warning=True` to disable this warning

</br>

### - epoch

In [24]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
print(num_training_steps)

1377


<br></br>

### - device

- 모델과 배치(batch)를 적재할 장치(device)를 정의

- GPU cuda or CPU

In [25]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

</br>

### - training

- 학습 루프 (Training Loop)

In [26]:
from tqdm import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        data = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**data)

        loss = outputs.loss #--> loss가 model에 encapsulation되어 있다.
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)

100%|█████████▉| 1376/1377 [02:46<00:00,  8.61it/s]

</br>

### - evaluation

- 평가 루프 (Evaluation Loop)

In [27]:
from datasets import load_metric

metric = load_metric("glue", "mrpc", trust_remote_code=True)
# FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets.
# Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
# FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric.
# You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
# You can avoid this message in future by passing the argument `trust_remote_code=True`.
# Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.

model.eval()
for batch in eval_dataloader:
    data = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**data)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    # torch.argmax() 함수는 주어진 텐서(tensor)에서 가장 큰 값의 인덱스를 반환하는 함수이다.
    # 즉, 텐서의 각 차원에 대해 최대값을 가지는 요소의 인덱스를 반환한다.
    # 딥러닝 모델의 출력에서 가장 확률이 높은 클래스를 예측하기 위해 사용된다.
    # 예를 들어, 이미지 분류 모델에서는 소프트맥스 활성화 함수를 통해 각 클래스에 대한 확률 분포를 얻은 후,
    # torch.nn.functional.softmax(logits, dim=-1)
    # torch.argmax() 함수를 사용하여 확률이 가장 높은 클래스의 인덱스를 예측한다.
    
    metric.add_batch(predictions=predictions, references=batch['labels'])

# metric.add_batch() 메서드로 평가 루프(evaluation loop)를 실행하면서 배치(batch)별 평가 메트릭(metrics) 계산 결과를 누적할 수 있다.
# 모든 배치(batch)를 누적하고 나면 metric.compute()로 최종 결과를 얻을 수 있다.
metric.compute()

# 1st. 학습 평가: {'accuracy': 0.8455882352941176, 'f1': 0.8919382504288165}
# 2nd. 학습 평가: {'accuracy': 0.8725490196078431, 'f1': 0.9100346020761245}

<ipython-input-27-4d99124d93b4>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc", trust_remote_code=True)


{'accuracy': 0.8725490196078431, 'f1': 0.9100346020761245}

In [ ]:
# # summary code

# from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# optimizer = AdamW(model.parameters(), lr=3e-5)

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )

# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

### - 전체 코드

In [ ]:
from datasets import load_dataset, load_metric
from transformers import (
    AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler
)
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


# 데이터 셋 적재
raw_datasets = load_dataset("glue", "mrpc")
# 사전학습 언어모델 checkpoint 이름 지정
checkpoint = "bert-base-uncased"
# 지정된 사전학습 언어모델에서 토크나이저 인스턴스화
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


# 토크나이저 함수 사용자 정의화 (sentence1, sentence2 컬럼에 대해서만 토크나이징 수행)
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# 토크나이징 수행
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# 배치(batch)별 패딩(padding)을 위한 data collator 정의
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 불필요한 입력 컬럼을 제거하고 사전학습 언어모델에 필요한 입력만 남김.
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# 데이터셋의 label 컬럼명을 labels로 변경
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# 데이터셋의 유형을 PyTorch tensor로 변경
tokenized_datasets.set_format("torch")

# 변경된 컬럼 출력
print(tokenized_datasets["train"].column_names)


# 각 종류별 데이터 로더 생성
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], shuffle=True, batch_size=8, collate_fn=data_collator)

# 사전학습 언어모델 인스턴스화
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# 최적화 함수 정의
optimizer = AdamW(model.parameters(), lr=5e-5)

# 에포크 개수 설정
num_epochs = 3
# 학습 스텝 수 계산
num_training_steps = num_epochs * len(train_dataloader)
# 학습 스케쥴러 설정
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# GPU로 모델을 이동
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 진행 상황바 정의
progress_bar = tqdm(range(num_training_steps))

# 모델을 학습 모드로 전환
model.train()
# 학습 루프 시작
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # 현재 배치 중에서 입력값을 모두 GPU로 이동.
        batch = {k: v.to(device) for k, v in batch.items()}
        # 모델 실행
        outputs = model(**batch)
        # 손실값 가져오기
        loss = outputs.loss
        # 역전파 수행
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# 평가 메트릭 가져오기
metric = load_metric("glue", "mrpc", trust_remote_code=True)
# 모델을 평가 모드로 전환
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

# 평가 결과 계산 및 출력
metric.compute()

-----

<br></br>

# Debugging

In [28]:
import transformers

transformers.__path__

['/usr/local/lib/python3.10/dist-packages/transformers']

In [ ]:
# 디버깅할 코드에 심을 수 있다.

# import pdb; pdb.set_trace()